# Imports

In [12]:
import math
import pandas as pd
import pennylane as qml
import time

from keras.datasets import mnist
from matplotlib import pyplot as plt
from pennylane import numpy as np
from pennylane.templates import AmplitudeEmbedding, AngleEmbedding
from pennylane.templates.subroutines import ArbitraryUnitary
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Model Params

In [13]:
np.random.seed(42)
initial_params = np.random.random([66])

INITIALIZATION_METHOD = 'Angle'
BATCH_SIZE = 20
EPOCHS = 400

STEP_SIZE = 0.01
BETA_1 = 0.9
BETA_2 = 0.99
EPSILON = 0.00000001

TRAINING_SIZE = 0.78
VALIDATION_SIZE = 0.07
TEST_SIZE = 1-TRAINING_SIZE-VALIDATION_SIZE

initial_time = time.time()

# Import dataset

In [14]:
(train_X, train_y), (test_X, test_y) = mnist.load_data()
examples = np.append(train_X, test_X, axis=0)
examples = examples.reshape(70000, 28*28)
classes = np.append(train_y, test_y)

In [15]:
x = []
y = []
for (example, label) in zip(examples, classes):
    if label == 0:
        x.append(example)
        y.append(-1)
    elif label == 1:
        x.append(example)
        y.append(1)

In [16]:
x = np.array(x)
y = np.array(y)

# Normalize pixels values
x = x / 255

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=TEST_SIZE, shuffle=True)

In [17]:
validation_indexes = np.random.random_integers(len(X_train), size=(math.floor(len(X_train)*VALIDATION_SIZE),))
X_validation = [X_train[n] for n in validation_indexes]
y_validation = [y_train[n] for n in validation_indexes]

In [18]:
pca = PCA(n_components=8)
pca.fit(X_train)
X_train = pca.transform(X_train)
X_validation = pca.transform(X_validation)
X_test = pca.transform(X_test)

preprocessing_time = time.time()

# Circuit creation

In [19]:
device = qml.device("default.qubit", wires=8)

In [20]:
def unitary(params, wire1, wire2):
    # qml.RZ(0, wires=wire1)
    qml.RY(params[0], wires=wire1)
    # qml.RZ(0, wires=wire1)
    # qml.RZ(0, wires=wire2)
    qml.RY(params[1], wires=wire2)
    # qml.RZ(0, wires=wire2)
    qml.CNOT(wires=[wire2, wire1])
    # qml.RZ(0, wires=wire1)
    qml.RY(params[2], wires=wire2)
    qml.CNOT(wires=[wire1, wire2])
    qml.RY(params[3], wires=wire2)
    qml.CNOT(wires=[wire2, wire1])
    # qml.RZ(0, wires=wire1)
    qml.RY(params[4], wires=wire1)
    # qml.RZ(0, wires=wire1)
    # qml.RZ(0, wires=wire2)
    qml.RY(params[5], wires=wire2)
    # qml.RZ(0, wires=wire2)

In [21]:
@qml.qnode(device)
def circuit(features, params):
    # Load state
    if INITIALIZATION_METHOD == 'Amplitude':
        AmplitudeEmbedding(features=features, wires=range(8), normalize=True, pad_with=0.)
    else:
        AngleEmbedding(features=features, wires=range(8), rotation='Y')

    # First layer
    unitary(params[0:6], 1, 2)
    unitary(params[6:12], 3, 4)
    unitary(params[12:18], 5, 6)

    # Second layer
    unitary(params[18:24], 0, 1)
    unitary(params[24:30], 2, 3)
    unitary(params[30:36], 4, 5)
    unitary(params[36:42], 6, 7)

    # Third layer
    unitary(params[42:48], 2, 5)

    # Fourth layer
    unitary(params[48:54], 1, 2)
    unitary(params[54:60], 5, 6)

    # Fifth layer
    unitary(params[60:66], 2, 5)

    # Measurement
    return qml.expval(qml.PauliZ(5))

## Circuit example

In [22]:
features = X_train[0]
print(f"Inital parameters: {initial_params}\n")
print(f"Example features: {features}\n")
print(f"Expectation value: {circuit(features, initial_params)}\n")
print(circuit.draw())

Inital parameters: [0.37454012 0.95071431 0.73199394 0.59865848 0.15601864 0.15599452
 0.05808361 0.86617615 0.60111501 0.70807258 0.02058449 0.96990985
 0.83244264 0.21233911 0.18182497 0.18340451 0.30424224 0.52475643
 0.43194502 0.29122914 0.61185289 0.13949386 0.29214465 0.36636184
 0.45606998 0.78517596 0.19967378 0.51423444 0.59241457 0.04645041
 0.60754485 0.17052412 0.06505159 0.94888554 0.96563203 0.80839735
 0.30461377 0.09767211 0.68423303 0.44015249 0.12203823 0.49517691
 0.03438852 0.9093204  0.25877998 0.66252228 0.31171108 0.52006802
 0.54671028 0.18485446 0.96958463 0.77513282 0.93949894 0.89482735
 0.59789998 0.92187424 0.0884925  0.19598286 0.04522729 0.32533033
 0.38867729 0.27134903 0.82873751 0.35675333 0.28093451 0.54269608]

Example features: [-3.49294491 -2.61674702 -1.40152433 -0.31434174  1.15175987 -0.22681643
  0.45016024 -0.16205067]

Expectation value: 0.5216955082088882

 0: ──RY(-3.49)───RY(0.432)──────────────────────────────────────────────────────────

# Accuracy test definition

In [23]:
def measure_accuracy(x, y, circuit_params):
    class_errors = 0

    for example, example_class in zip(x, y):
        predicted_value = circuit(example, circuit_params)

        if (example_class > 0 and predicted_value <= 0) or (example_class <= 0 and predicted_value > 0):
            class_errors += 1

    return 1 - (class_errors/len(y))

# Training

In [24]:
params = initial_params
opt = qml.AdamOptimizer(stepsize=STEP_SIZE, beta1=BETA_1, beta2=BETA_2, eps=EPSILON)
test_accuracies = []
best_validation_accuracy = 0.0
best_params = []

for i in range(len(X_train)):
    features = X_train[i]
    expected_value = y_train[i]

    def cost(circuit_params):
        value = circuit(features, circuit_params)
        return ((expected_value - value) ** 2)/len(X_train)

    params = opt.step(cost, params)

    if i % BATCH_SIZE == 0:
        print(f"epoch {i//BATCH_SIZE}")

    if i % (10*BATCH_SIZE) == 0:
        current_accuracy = measure_accuracy(X_validation, y_validation, params)
        test_accuracies.append(current_accuracy)
        print(f"accuracy: {current_accuracy}")

        if current_accuracy > best_validation_accuracy:
            print("best accuracy so far!")
            best_validation_accuracy = current_accuracy
            best_params = params

    if len(test_accuracies) == 30:
        print(f"test_accuracies: {test_accuracies}")

        if np.allclose(best_validation_accuracy, test_accuracies[0]):
            params = best_params
            break

        del test_accuracies[0]

epoch 0
accuracy: 0.7474402730375427
best accuracy so far!
epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch 8
epoch 9
epoch 10
accuracy: 0.8031854379977247
best accuracy so far!
epoch 11
epoch 12
epoch 13
epoch 14
epoch 15
epoch 16
epoch 17
epoch 18
epoch 19
epoch 20
accuracy: 0.8316268486916951
best accuracy so far!
epoch 21
epoch 22
epoch 23
epoch 24
epoch 25
epoch 26
epoch 27
epoch 28
epoch 29
epoch 30
accuracy: 0.8293515358361775
epoch 31
epoch 32
epoch 33
epoch 34
epoch 35
epoch 36
epoch 37
epoch 38
epoch 39
epoch 40
accuracy: 0.856655290102389
best accuracy so far!
epoch 41
epoch 42
epoch 43
epoch 44
epoch 45
epoch 46
epoch 47
epoch 48
epoch 49
epoch 50
accuracy: 0.8748577929465302
best accuracy so far!
epoch 51
epoch 52
epoch 53
epoch 54
epoch 55
epoch 56
epoch 57
epoch 58
epoch 59
epoch 60
accuracy: 0.850967007963595
epoch 61
epoch 62
epoch 63
epoch 64
epoch 65
epoch 66
epoch 67
epoch 68
epoch 69
epoch 70
accuracy: 0.8555176336746303
epoch 71
epoch 72
epoch 73
epoc

epoch 341
epoch 342
epoch 343
epoch 344
epoch 345
epoch 346
epoch 347
epoch 348
epoch 349
epoch 350
accuracy: 0.875995449374289
test_accuracies: [0.850967007963595, 0.8555176336746303, 0.8555176336746303, 0.8350398179749716, 0.850967007963595, 0.856655290102389, 0.8498293515358362, 0.8612059158134243, 0.875995449374289, 0.8839590443686007, 0.8680318543799772, 0.8873720136518771, 0.8839590443686007, 0.8919226393629124, 0.8930602957906713, 0.8919226393629124, 0.8748577929465302, 0.881683731513083, 0.8748577929465302, 0.8771331058020477, 0.8805460750853242, 0.8896473265073948, 0.8976109215017065, 0.8839590443686007, 0.8953356086461889, 0.9010238907849829, 0.8919226393629124, 0.8964732650739476, 0.8805460750853242, 0.875995449374289]
epoch 351
epoch 352
epoch 353
epoch 354
epoch 355
epoch 356
epoch 357
epoch 358
epoch 359
epoch 360
accuracy: 0.8873720136518771
test_accuracies: [0.8555176336746303, 0.8555176336746303, 0.8350398179749716, 0.850967007963595, 0.856655290102389, 0.8498293515358

epoch 454
epoch 455
epoch 456
epoch 457
epoch 458
epoch 459
epoch 460
accuracy: 0.8930602957906713
test_accuracies: [0.8873720136518771, 0.8839590443686007, 0.8919226393629124, 0.8930602957906713, 0.8919226393629124, 0.8748577929465302, 0.881683731513083, 0.8748577929465302, 0.8771331058020477, 0.8805460750853242, 0.8896473265073948, 0.8976109215017065, 0.8839590443686007, 0.8953356086461889, 0.9010238907849829, 0.8919226393629124, 0.8964732650739476, 0.8805460750853242, 0.875995449374289, 0.8873720136518771, 0.8805460750853242, 0.8521046643913538, 0.870307167235495, 0.8782707622298066, 0.8998862343572241, 0.8919226393629124, 0.8987485779294653, 0.8896473265073948, 0.8850967007963595, 0.8930602957906713]
epoch 461
epoch 462
epoch 463
epoch 464
epoch 465
epoch 466
epoch 467
epoch 468
epoch 469
epoch 470
accuracy: 0.8873720136518771
test_accuracies: [0.8839590443686007, 0.8919226393629124, 0.8930602957906713, 0.8919226393629124, 0.8748577929465302, 0.881683731513083, 0.8748577929465302, 

epoch 566
epoch 567
epoch 568
epoch 569
epoch 570
accuracy: 0.8896473265073948
test_accuracies: [0.8976109215017065, 0.8839590443686007, 0.8953356086461889, 0.9010238907849829, 0.8919226393629124, 0.8964732650739476, 0.8805460750853242, 0.875995449374289, 0.8873720136518771, 0.8805460750853242, 0.8521046643913538, 0.870307167235495, 0.8782707622298066, 0.8998862343572241, 0.8919226393629124, 0.8987485779294653, 0.8896473265073948, 0.8850967007963595, 0.8930602957906713, 0.8873720136518771, 0.8782707622298066, 0.8907849829351536, 0.9010238907849829, 0.888509670079636, 0.8976109215017065, 0.8782707622298066, 0.8987485779294653, 0.8964732650739476, 0.8828213879408419, 0.8896473265073948]
epoch 571
epoch 572
epoch 573
epoch 574
epoch 575
epoch 576
epoch 577
epoch 578
epoch 579
epoch 580
accuracy: 0.8953356086461889
test_accuracies: [0.8839590443686007, 0.8953356086461889, 0.9010238907849829, 0.8919226393629124, 0.8964732650739476, 0.8805460750853242, 0.875995449374289, 0.8873720136518771, 

In [25]:
print("Optimized rotation angles: {}".format(params))

training_time = time.time()

Optimized rotation angles: [-9.04859010e-02  5.21318104e-01  1.21340561e+00  5.46227343e-01
 -7.40838500e-01 -3.84680424e-01  1.91362477e-01 -1.10062378e-03
  4.94078673e-02  3.63193194e-01  2.62474785e-01  1.26018906e+00
  2.38449411e-01  1.01839313e+00 -1.43157845e-04  4.94886174e-01
  5.85968653e-01  4.98658941e-01  2.16866479e-01 -6.05628000e-01
 -1.01915210e-01  2.19002754e-01  2.92144649e-01  2.08855283e-01
 -8.46049597e-02  1.02706625e+00  2.51809118e-02  6.52715501e-03
  8.94196876e-01  4.64504127e-02  8.97824057e-01  4.52250534e-01
 -9.27023499e-02  3.38897743e-01  9.65632033e-01  1.14882029e+00
  2.78516278e-01 -1.44688150e-01  1.48202877e+00 -1.70858632e+00
 -5.06776342e-01  4.95176910e-01  3.36170829e-01  1.24974335e+00
 -5.28974323e-01  1.64255964e+00  5.92079840e-01  8.47226441e-01
  3.89203719e-01  4.65223219e-01  1.55727499e+00  1.69994674e+00
  9.39498942e-01  1.61156768e+00  9.25058398e-01  2.93059658e-01
  1.87683268e+00  1.43216581e+00  1.65046768e-01  3.25330331e-0

# Testing

In [26]:
accuracy = measure_accuracy(X_test, y_test, params)
print(accuracy)

test_time = time.time()

0.8687415426251691


In [27]:
print(f"pre-processing time: {preprocessing_time-initial_time}")
print(f"training time: {training_time - preprocessing_time}")
print(f"test time: {test_time - training_time}")
print(f"total time: {test_time - initial_time}")

pre-processing time: 2.692225694656372
training time: 1869.1098012924194
test time: 50.144665002822876
total time: 1921.9466919898987
